In [114]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import re


In [115]:
path='//192.168.14.50/Public/ВЫГРУЗКИ/'
files=os.listdir(path)


In [116]:

def shop_name(shop):

    m1 = re.compile('([0-9]{2}-[0-9]{,2}-)([0-9]{,3})')
    m = m1.findall(shop)
    for i in range(len(m)):
        m[i] = m[i][0] + ('000' + m[i][1])[-3:]
    return m[0]

In [117]:

ds=pd.read_csv(path + 'ПРОДАЖИ ПО ДНЯМ (таблица для анализа) от 04-09-24.txt', sep='\t')
ds['Выручка,']=ds['Выручка,'].str.replace(',','.')
ds['Выручка,']=ds['Выручка,'].str.replace(chr(160),'')
ds['Количество']=ds['Количество'].str.replace(',','.')
ds['Выручка,']=pd.to_numeric(ds['Выручка,'])
ds['День.Начало дня']=pd.to_datetime(ds['День.Начало дня'],dayfirst=True,errors='coerce')
ds=ds[ds['День.Начало дня'].isna()==False]
ds=ds.assign(Номер_тт= ds['Склад / магазин'].apply(shop_name))
ds[['key','product']]=ds['Код, Номенклатура'].str.split(',',n=1,expand=True)
ds['Выручка,'].fillna(0)
ds['Количество']=pd.to_numeric(ds['Количество'])
ds= ds.dropna(subset=['Количество'])
ds=ds.drop(['Код, Номенклатура'],axis=1)
ds['weekday']= pd.DatetimeIndex(ds['День.Начало дня']).weekday
ds['day']= pd.DatetimeIndex(ds['День.Начало дня']).day
ds['month']= pd.DatetimeIndex(ds['День.Начало дня']).month
ds['year']= pd.DatetimeIndex(ds['День.Начало дня']).year


In [118]:
ds.columns=['date', 'shop_name', 'person', 'feature',
       'amount', 'sales', 'shop_id', 'key', 'product','weekday','day','month', 'year'       ]

In [119]:
ds.describe(include='all')

,date,shop_name,person,feature,amount,sales,shop_id,key,product,weekday,day,month,year
count,89839,89839,89825,44180,89839.000000,89839.000000,89839,89839,89839,89839.000000,89839.000000,89839.0,89839.0
unique,NaN,194,392,68,NaN,NaN,194,970,970,NaN,NaN,NaN,NaN
top,NaN,"64-3-092 Саратов,пл.Орджоникидзе, 1(Оранжевый)...",Баркова Ирина Анатольевна,70x50,NaN,NaN,64-3-092,НФ-00001439,Пакет Майка,NaN,NaN,NaN,NaN
freq,NaN,1421,803,5825,NaN,NaN,1421,2701,2701,NaN,NaN,NaN,NaN
mean,2024-08-16 02:27:47.062189312,NaN,NaN,NaN,1.446042,704.545209,NaN,NaN,NaN,3.052928,16.102628,8.0,2024.0
min,2024-08-01 00:00:00,NaN,NaN,NaN,-4.000000,-5299.000000,NaN,NaN,NaN,0.000000,1.000000,8.0,2024.0
25%,2024-08-09 00:00:00,NaN,NaN,NaN,1.000000,199.000000,NaN,NaN,NaN,1.000000,9.000000,8.0,2024.0
50%,2024-08-16 00:00:00,NaN,NaN,NaN,1.000000,458.000000,NaN,NaN,NaN,3.000000,16.000000,8.0,2024.0
75%,2024-08-23 00:00:00,NaN,NaN,NaN,1.000000,958.000000,NaN,NaN,NaN,5.000000,23.000000,8.0,2024.0
max,2024-08-31 00:00:00,NaN,NaN,NaN,40.000000,10990.000000,NaN,NaN,NaN,6.000000,31.000000,8.0,2024.0


In [120]:
ds_prod=ds[['product','key','feature']].drop_duplicates().reset_index(drop=True)
ds_shops=ds[['shop_name']].drop_duplicates().reset_index(drop=True)
ds=ds.reset_index(drop=True)

In [121]:
daily_per_shop=ds.groupby(['shop_id','weekday','day','month','year'],as_index=False).agg({'key':['count'],'amount':['sum','mean','median','std'],'sales':['sum','mean','median','std']})
daily_per_shop.columns=[ '_'.join(x) for x in daily_per_shop.columns]


In [122]:
daily_per_shop

,shop_id_,weekday_,day_,month_,year_,key_count,amount_sum,amount_mean,amount_median,amount_std,sales_sum,sales_mean,sales_median,sales_std
0,18-3-000,0,5,8,2024,5,10.0,2.000000,2.0,0.707107,624.0,124.800000,99.0,72.074267
1,18-3-000,0,12,8,2024,3,3.0,1.000000,1.0,0.000000,3027.0,1009.000000,559.0,1228.454313
2,18-3-000,1,6,8,2024,2,2.0,1.000000,1.0,0.000000,714.0,357.000000,357.0,483.661038
3,18-3-000,1,13,8,2024,3,4.0,1.333333,1.0,0.577350,3906.0,1302.000000,1779.0,842.696268
4,18-3-000,2,14,8,2024,5,7.0,1.400000,1.0,0.547723,5104.0,1020.800000,1349.0,932.945175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5832,73-3-352,5,31,8,2024,13,20.0,1.538462,1.0,1.126601,9996.0,768.923077,369.0,775.254309
5833,73-3-352,6,4,8,2024,19,28.0,1.473684,1.0,0.772328,12863.0,677.000000,549.0,583.004669
5834,73-3-352,6,11,8,2024,12,14.0,1.166667,1.0,0.389249,7478.0,623.166667,439.0,600.565314
5835,73-3-352,6,18,8,2024,14,14.0,1.000000,1.0,0.000000,15166.0,1083.285714,639.0,910.475060
